In [1]:
import pandas as pd
import numpy as np

In [14]:
def split_df(dataframe):
    total_rows = len(dataframe)
    train_end = int(total_rows * 0.6)
    validation_end = train_end + int(total_rows * 0.2)

    dataframe['split'] = 'train'  # Default to 'train'
    dataframe.loc[train_end:validation_end-1, 'split'] = 'validation'
    dataframe.loc[validation_end:, 'split'] = 'test'
    return dataframe

# Load

In [2]:
read_dir = './processed/'

In [3]:
cohort = pd.read_csv(read_dir + 'cohort.csv', index_col=0)
diag = pd.read_csv(read_dir + 'diag.csv', index_col=0)
labs = pd.read_csv(read_dir + 'labs.csv', index_col=0)
proc = pd.read_csv(read_dir + 'proc.csv', index_col=0)
meds = pd.read_csv(read_dir + 'meds.csv', index_col=0)

# Clean

In [55]:
save_dir = './cleaned/'

## Cohort

In [56]:
cohort_new = cohort.copy()

cohort_new['seqnum'] = cohort_new.groupby('subject_id').cumcount()

cohort_new['admit_1'] = cohort_new.groupby('subject_id')['admittime'].apply(lambda i:i.shift(1))
cohort_new['difftime'] = pd.to_datetime(cohort_new['admittime']) - pd.to_datetime(cohort_new['admit_1'])
cohort_new['diffmonth'] = cohort_new['difftime'].apply(lambda x: x.days/30)

cohort_new['male'] = (cohort_new.gender=='M').astype(float)
cohort_new['white'] = (cohort_new.ethnicity=='WHITE').astype(float)
cohort_new['medicare'] = (cohort_new.insurance=='Medicare').astype(float)
cohort_new['medicaid'] = (cohort_new.insurance=='Medicaid').astype(float)

cohort_new = cohort_new[['subject_id', 'hadm_id', 'seqnum', 'diffmonth', 
                         'male', 'white', 'medicare', 'medicaid',
                         'label_hf','label_diabetes']].fillna(0)

cohort_sub = cohort_new.copy()
cohort_sub = cohort_sub[['subject_id','male','white','medicare','medicaid','label_hf','label_diabetes']]
cohort_sub = cohort_sub.groupby('subject_id', as_index=False).max()
split_dfs = []
for male in [0, 1]:
    for white in [0, 1]:
        for medicare in [0, 1]:
            for medicaid in [0, 1]:
                for label_hf in [0, 1]:
                    for label_diabetes in [0, 1]:
                        subset = cohort_sub[(cohort_sub['male'] == male) & 
                                            (cohort_sub['white'] == white) & 
                                            (cohort_sub['medicare'] == medicare) & 
                                            (cohort_sub['medicaid'] == medicaid) &
                                            (cohort_sub['label_hf'] == label_hf) &
                                            (cohort_sub['label_diabetes'] == label_diabetes)]
                        if not subset.empty:
                            split_dfs.append(split_df(subset.copy().reset_index()))
cohort_sub = pd.concat(split_dfs).sort_values(['subject_id']).reset_index()                
cohort_sub = cohort_sub.drop(columns=['level_0', 'index'])
print(cohort_sub.split.value_counts())

cohort_new = cohort_new.merge(cohort_sub[['subject_id','split']])
for suffix in ['hf', 'diabetes']:
    label = 'label_' + suffix
    split = 'split_' + suffix
    cohort_new[split] = pd.NA
    label_0 = cohort_sub[cohort_sub[label]==0].subject_id.to_numpy()
    cohort_0 = cohort_new.copy().loc[(cohort_new.subject_id.isin(label_0)), ['subject_id','hadm_id','seqnum']]
    cohort_0 = cohort_0.sort_values(by=['subject_id', 'seqnum'], ascending=[True, False])
    cohort_1 = cohort_new.copy().loc[cohort_new[label]==1, ['subject_id','hadm_id','seqnum']]
    cohort_1 = cohort_1.sort_values(by=['subject_id', 'seqnum'], ascending=[True, True])
    cohort_01 = pd.concat([cohort_0, cohort_1])
    cohort_01 = cohort_01.drop_duplicates(subset=['subject_id'], keep='first')
    hadm_01 = cohort_01.hadm_id.to_numpy()
    cohort_new.loc[cohort_new.hadm_id.isin(hadm_01), split] = cohort_new.loc[cohort_new.hadm_id.isin(hadm_01), 'split']

cohort_new = cohort_new.drop(columns=['split'])
print(cohort_new.split_hf.value_counts())
print(cohort_new.split_diabetes.value_counts())
cohort_new[:20]

train         23532
test           7908
validation     7825
Name: split, dtype: int64
train         23532
test           7908
validation     7825
Name: split_hf, dtype: int64
train         23532
test           7908
validation     7825
Name: split_diabetes, dtype: int64


,subject_id,hadm_id,seqnum,diffmonth,male,white,medicare,medicaid,label_hf,label_diabetes,split_hf,split_diabetes
0,10000635.0,26134563.0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,train,train
1,10000980.0,29654838.0,0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,<NA>,<NA>
2,10000980.0,26913865.0,1,18.000000,0.0,0.0,1.0,0.0,0.0,0.0,<NA>,<NA>
3,10000980.0,24947999.0,2,16.566667,0.0,0.0,1.0,0.0,0.0,0.0,<NA>,<NA>
4,10000980.0,25242409.0,3,4.900000,0.0,0.0,1.0,0.0,0.0,0.0,<NA>,<NA>
5,10000980.0,25911675.0,4,1.633333,0.0,0.0,1.0,0.0,0.0,0.0,<NA>,<NA>
6,10000980.0,29659838.0,5,1.766667,0.0,0.0,1.0,0.0,0.0,0.0,<NA>,<NA>
7,10000980.0,20897796.0,6,25.333333,0.0,0.0,0.0,0.0,0.0,0.0,train,train
8,10001176.0,23334588.0,0,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,train,train
9,10001667.0,22672901.0,0,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,train,train


In [57]:
cohort_new.to_csv(save_dir + 'cohort.csv')
cohort_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108807 entries, 0 to 108806
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   subject_id      108807 non-null  float64
 1   hadm_id         108807 non-null  float64
 2   seqnum          108807 non-null  int64  
 3   diffmonth       108807 non-null  float64
 4   male            108807 non-null  float64
 5   white           108807 non-null  float64
 6   medicare        108807 non-null  float64
 7   medicaid        108807 non-null  float64
 8   label_hf        108807 non-null  float64
 9   label_diabetes  108807 non-null  float64
 10  split_hf        39265 non-null   object 
 11  split_diabetes  39265 non-null   object 
dtypes: float64(9), int64(1), object(2)
memory usage: 10.8+ MB


## Diag

In [145]:
diag_new = diag.copy()

diag_kept = dict(diag_new[['subject_id', 'new_icd_code']].drop_duplicates().new_icd_code.value_counts() > 
                 0.01 * len(cohort_new.subject_id.unique()))
diag_kept = [key for key, value in diag_kept.items() if value]
diag_new = diag_new[diag_new.new_icd_code.isin(diag_kept)]

diag_new['value'] = 1
diag_new = diag_new.pivot_table(index='hadm_id', columns='new_icd_code', values='value').fillna(0).reset_index()

diag_new = cohort_new[['hadm_id', 'diffmonth']].merge(diag_new)

diag_new.sample(10)

,hadm_id,diffmonth,A04,A40,A41,B15,B19,B35,B37,B95,...,Z89,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z98,Z99
20141,20709265.0,7.933333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90411,29675904.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6013,23269947.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87239,25977354.0,1.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
71360,26699482.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
91629,29488289.0,9.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
62820,22409705.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90466,21266903.0,0.633333,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20844,26608015.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52031,25222906.0,2.566667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
diag_new.to_csv(save_dir + 'diag.csv')
diag_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108807 entries, 0 to 108806
Columns: 307 entries, hadm_id to Z99
dtypes: float64(307)
memory usage: 255.7 MB


## Labs

In [160]:
labs_new = labs.copy()

labs_kept = dict(labs_new[['subject_id', 'itemid']].drop_duplicates().itemid.value_counts() > 
                 0.1 * len(cohort_new.subject_id.unique()))
labs_kept = [key for key, value in labs_kept.items() if value]
labs_new = labs_new[labs_new.itemid.isin(labs_kept)]

labs_new['hour'] = labs_new.lab_time_from_admit.apply(lambda x: pd.to_timedelta(x).total_seconds()/3600)
labs_new = labs_new.sort_values(['subject_id','hadm_id','hour']).reset_index()
labs_new = labs_new[['hadm_id','itemid','hour','valuenum']]

labs_new[:100]

labs_new = labs_new.groupby(['hadm_id','hour','itemid'], as_index=False).agg({'valuenum':'mean'})
labs_new = labs_new.pivot(index=['hadm_id','hour'],columns='itemid',values='valuenum').reset_index()
labs_new.update(labs_new.groupby('hadm_id').fillna(method='ffill'))
labs_new = labs_new.fillna(labs_new.median())

labs_new[:100]

itemid,hadm_id,hour,50802,50804,50806,50808,50809,50810,50811,50813,...,51514,51516,51613,51678,52069,52073,52074,52075,52135,52172
0,20000034.0,12.366667,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,5.0,0.03,0.26,0.68,9.55,0.6,49.3
1,20000034.0,15.783333,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,115.0,140.0,5.0,0.03,0.26,0.68,9.55,0.6,49.3
2,20000034.0,35.633333,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,115.0,140.0,7.0,0.19,0.25,0.19,4.91,0.6,50.9
3,20000041.0,15.133333,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,8.0,0.03,0.09,0.70,7.08,0.6,50.9
4,20000041.0,23.750000,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,8.0,0.03,0.09,0.70,7.08,0.6,50.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20000471.0,-5.066667,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,8.0,0.03,0.09,0.70,7.08,0.6,50.9
96,20000471.0,13.100000,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,8.0,0.03,0.09,0.70,7.08,0.6,50.9
97,20000471.0,36.850000,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,8.0,0.03,0.09,0.70,7.08,0.6,50.9
98,20000471.0,47.850000,0.0,26.0,104.0,1.12,137.0,29.0,9.7,1.5,...,2.0,4.0,140.0,8.0,0.03,0.09,0.70,7.08,0.6,50.9


In [161]:
labs_new.to_csv(save_dir + 'labs.csv')
labs_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360987 entries, 0 to 1360986
Columns: 108 entries, hadm_id to 52172
dtypes: float64(108)
memory usage: 1.1 GB


## Proc

In [129]:
proc_new = proc.copy()

proc_kept = dict(proc_new[['subject_id', 'icd_code']].drop_duplicates().icd_code.value_counts() > 
                 0.01 * len(cohort_new.subject_id.unique()))
proc_kept = [key for key, value in proc_kept.items() if value]
proc_new = proc_new[proc_new.icd_code.isin(proc_kept)]

proc_new['hour'] = proc_new.proc_time_from_admit.apply(lambda x: pd.to_timedelta(x).total_seconds()/3600)
proc_new = proc_new.sort_values(['subject_id','hadm_id','hour']).reset_index()
proc_new = proc_new[['subject_id','hadm_id','icd_code','hour']]

proc_new['value'] = 1
proc_new = proc_new.pivot_table(index=['hadm_id','hour'], columns='icd_code', values='value').fillna(0).reset_index()

proc_new[:100]

icd_code,hadm_id,hour,02100Z9,02H633Z,02HV33Z,06BQ4ZZ,0BH17EZ,0DJ08ZZ,0DJD8ZZ,3E0G76Z,4A023N6,4A023N7,5A1221Z,5A1945Z,5A1955Z,5A1D60Z,5A1D70Z,5A2204Z,B211YZZ
0,20000147,-16.550000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,20000239,-16.350000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,20002054,28.783333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20002267,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20003095,342.533333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20067636,731.150000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,20067999,81.133333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97,20068675,34.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
98,20070316,296.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [130]:
proc_new.to_csv(save_dir + 'proc.csv')
proc_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13708 entries, 0 to 13707
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   hadm_id  13708 non-null  int64  
 1   hour     13708 non-null  float64
 2   02100Z9  13708 non-null  float64
 3   02H633Z  13708 non-null  float64
 4   02HV33Z  13708 non-null  float64
 5   06BQ4ZZ  13708 non-null  float64
 6   0BH17EZ  13708 non-null  float64
 7   0DJ08ZZ  13708 non-null  float64
 8   0DJD8ZZ  13708 non-null  float64
 9   3E0G76Z  13708 non-null  float64
 10  4A023N6  13708 non-null  float64
 11  4A023N7  13708 non-null  float64
 12  5A1221Z  13708 non-null  float64
 13  5A1945Z  13708 non-null  float64
 14  5A1955Z  13708 non-null  float64
 15  5A1D60Z  13708 non-null  float64
 16  5A1D70Z  13708 non-null  float64
 17  5A2204Z  13708 non-null  float64
 18  B211YZZ  13708 non-null  float64
dtypes: float64(18), int64(1)
memory usage: 2.0 MB


## Meds

In [131]:
meds.sample(5)

,subject_id,hadm_id,drug_name,start_hours_from_admit,stop_hours_from_admit,dose_val_rx,label_hf,label_diabetes
2738524,19674244,28478629,vancomycin hydrochloride,23 days 18:41:00,33 days 16:41:00,125,1.0,1.0
1684028,15881144,21021706,propofol,0 days 07:45:00,1 days 00:45:00,1000,0.0,0.0
549650,11954455,25843504,sodium chloride,0 days 03:45:00,0 days 02:45:00,1000,0.0,0.0
583554,12065081,28534640,furosemide,2 days 16:26:00,2 days 21:26:00,80,0.0,0.0
2624809,19270996,21757609,aspirin,-1 days +16:03:00,1 days 04:03:00,81,0.0,0.0


In [140]:
meds_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135221 entries, 0 to 2135220
Data columns (total 5 columns):
 #   Column       Dtype  
---  ------       -----  
 0   subject_id   int64  
 1   hadm_id      int64  
 2   drug_name    object 
 3   hour         float64
 4   dose_val_rx  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 81.5+ MB


In [141]:
meds_new = meds.copy()

meds_kept = dict(meds_new[['subject_id', 'drug_name']].drop_duplicates().drug_name.value_counts() > 
                 0.1 * len(cohort_new.subject_id.unique()))
meds_kept = [key for key, value in meds_kept.items() if value]
meds_new = meds_new[meds_new.drug_name.isin(meds_kept)]

meds_new['hour'] = meds_new.start_hours_from_admit.apply(lambda x: pd.to_timedelta(x).total_seconds()/3600)
meds_new = meds_new.sort_values(['subject_id','hadm_id','hour']).reset_index()
meds_new['value'] = meds_new.dose_val_rx.apply(pd.to_numeric, errors='coerce').dropna()
meds_new = meds_new[['subject_id','hadm_id','drug_name','hour','value']]

meds_new = meds_new.pivot_table(index=['hadm_id','hour'], columns='drug_name', values='value').fillna(0).reset_index()

meds_new[:100]

drug_name,hadm_id,hour,acetaminophen,albuterol sulfate,"aluminum hydroxide, magnesium hydroxide, and simethicone",aspirin,atorvastatin calcium,bisacodyl,carvedilol,cefazolin sodium,...,polyethylene glycol 3350,potassium chloride,"potassium chloride, dextrose monohydrate and sodium chloride",prednisone,propofol,sennosides,sodium chloride,"sodium chloride, potassium chloride, sodium lactate and calcium chloride",tramadol hydrochloride,vancomycin hydrochloride
0,20000034,2.783333,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.6,0.0,1000.0,0.0,0.0
1,20000034,3.783333,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20000034,5.783333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20000034,12.783333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.6,0.0,0.0,0.0,0.0
4,20000034,26.783333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20001135,1.683333,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0,0.0
96,20001135,4.683333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,20001135,6.683333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,20001135,8.683333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
meds_new.to_csv(save_dir + 'meds.csv')
meds_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1216815 entries, 0 to 1216814
Data columns (total 56 columns):
 #   Column                                                                    Non-Null Count    Dtype  
---  ------                                                                    --------------    -----  
 0   hadm_id                                                                   1216815 non-null  int64  
 1   hour                                                                      1216815 non-null  float64
 2   acetaminophen                                                             1216815 non-null  float64
 3   albuterol sulfate                                                         1216815 non-null  float64
 4   aluminum hydroxide, magnesium hydroxide, and simethicone                  1216815 non-null  float64
 5   aspirin                                                                   1216815 non-null  float64
 6   atorvastatin calcium                      

In [58]:
!tar -zcvf cleaned.tar.gz cleaned

cleaned/
cleaned/diag.csv
cleaned/.ipynb_checkpoints/
cleaned/proc.csv
cleaned/meds.csv
cleaned/labs.csv
cleaned/cohort.csv
